# Notebook Description:

The following notebook contains code for processing and cropping .svs files from whole slide images down to 512x512 crops that will be inputted to the trained DLC model for classification.

- For TCGA samples the .svs files were downloaded from: https://portal.gdc.cancer.gov/projects/TCGA-THCA

- For Nikiforov samples the .svs files were initially processed by accessing the web server svs files hosted on Aperio Image Scope through: http://image.upmc.edu:8080/NikiForov%20EFV%20Study/BoxA/view.apml?listview=1. A script was written to split the image in parralel before saving each crop for futher processing in this notebook.

For both datasets, crops of 512x512 were extracted from the whole slide images and then filtered to only contain images patches where the mean and standard deviation of the pixel values were one standard deviation away from that of the TharunThompson reference dataset. 

From this subset, a random sample of 20 crops was selected to be submitted to the trained classifier for testing.


In [1]:
import glob, os, random, tqdm, shutil
from tqdm import tqdm
import numpy as np
import openslide
from openslide import OpenSlideError
from openslide.deepzoom import DeepZoomGenerator
from PIL import Image

# Find Mean and Std Pixel Values of Reference Dataset:

In [2]:
def find_mean_std_pixel_value(img_list, image_type):
    """
    Finds the mean and std pixel values for a list of images.
    """   
    avg_pixel_value = []
    stddev_pixel_value= []
    for file in img_list:
        image = Image.open(file)
        img_arr = np.array(image)
        avg = img_arr.mean()
        std = img_arr.std()
        avg_pixel_value.append(avg)
        stddev_pixel_value.append(std)
        
    avg_pixel_value = np.array(avg_pixel_value)  
    stddev_pixel_value = np.array(stddev_pixel_value)
        
    print("Average pixel value for %s images is:" % image_type, 
          avg_pixel_value.mean())
    print("Std of mean values is:", 
          avg_pixel_value.std())
    print("Average std dev of pixel value for %s images is:" % image_type, 
          stddev_pixel_value.mean())
    print("Std of std values is:", 
          stddev_pixel_value.std())
    
    return avg_pixel_value.mean(), avg_pixel_value.std(), \
            stddev_pixel_value.mean(), stddev_pixel_value.std()

In [3]:
# Getting an image list for the TharunThompson 512x512 cropped images:
tt_imgs = (glob.glob("all_imgs/TT_Cond/*.png"))
tt_imgs[0:5]

['all_imgs/TT_Cond\\img00000000.png',
 'all_imgs/TT_Cond\\img00000001.png',
 'all_imgs/TT_Cond\\img00000002.png',
 'all_imgs/TT_Cond\\img00000003.png',
 'all_imgs/TT_Cond\\img00000004.png']

In [47]:
tt_mean, tt_mean_std, tt_std, tt_std_std = find_mean_std_pixel_value(
                                              tt_imgs, 'Tharun Thompson')

Average pixel value for Tharun Thompson images is: 150.08757740333542
Std of mean values is: 23.83420922062602
Average std dev of pixel value for Tharun Thompson images is: 49.310766475752004
Std of std values is: 6.027017945351431


In [ ]:
tt_mean, tt_mean_std, tt_std, tt_std_std = 150.0875, 23.8342, 49.3107, 6.027

# Loading and Cropping Whole Slide Images:

In [5]:
def load_slide(filename):
    """
    Loads a whole-slide image (*.svs, etc).
    Args:
    filename: Name of the slide file.
    Returns:
    An OpenSlide object representing a whole-slide image.
    """
    try:
        slide = openslide.open_slide(filename)
    except OpenSlideError:
        slide = None
    except FileNotFoundError:
        slide = None
    
    return slide

In [6]:
def crop_wsi(slide_path, crop_size, tile_level, 
             tile_mean, tile_mean_std, tile_std, tile_std_std, 
             save_loc, sample_no):
    """
    slide: the whole slide image
    crop_size: the size of each crop to perform
    tile_level: the level of Deep Zoom corresponding to the resolution desired
    tile_mean: image mean pixel value from reference dataset
    tile_mean_std: std of the tile means
    tile_std: image std of pixel value from reference dataset
    tile_std_std: std of the tile stds
    sample_no: no. of sample slides to return
    """
     # Return file id from path:
    file_id = os.path.splitext((os.path.basename(slide_path)))[0]

    # Load the slide:
    try:
        slide = load_slide(slide_path)
    except:
        print("Corrupt slide: %s." % (file_id))
        return file_id

    # Create a Deep Zoom Generator Object with crop size = tile size:
    tiles = DeepZoomGenerator(slide, 
                              tile_size = crop_size, 
                              overlap=0,        # zero overlap between crops
                              limit_bounds=False)
    
    # Looking at a specific zoom level:
    level_num = min(16, tiles.level_count-1)  

    # Extract the number of cols/rows in specified tile level of Deep Zoom:
    cols, rows = tiles.level_tiles[level_num]

    # Create a list of all tiles that match search criteria (mean/std)
    #print("Creating tile list...")
    cr_list = []

    for row in tqdm(range(rows), desc = 'Row Processing Progress'):
        for col in range(cols):
            temp_tile = tiles.get_tile(level_num, (col, row))
            temp_tile_RGB = temp_tile.convert('RGB')
            temp_tile_np = np.array(temp_tile_RGB)
            # Calc. pixel mean and std of tile:
            t_m = temp_tile_np.mean()
            t_s = temp_tile_np.std()

            # Setting limits equal to reference dataset 1 std range:
            if (t_m > tt_mean - tt_mean_std) and (t_m < tt_mean + tt_mean_std)\
                and (t_s > tt_std - tt_std_std) and (t_s < tt_std + tt_std_std):
                cr_list.append((col, row))

    print("Number of Crops Matching Criteria: ", len(cr_list))

    # Selecting random crops = sample_no
    sample_no = min(len(cr_list), sample_no)
    sel_crops = random.sample(cr_list, sample_no)

    # Saving the crops:
    for i in range(len(sel_crops)):
        # Return the cropped image for each saved tile: 
        crop = tiles.get_tile(
                        level_num, 
                        sel_crops[i]) # Deep zoom level and address (column, row)

        # Convert to RGB and save:
        crop_RGB = crop.convert('RGB')
        crop_RGB.save("%s/%s_%s.jpeg" % (save_loc, file_id, i))
    
    print("Saved %s cropped images from %s." % (sample_no, file_id))

# Processing TCGA Slides:
- Processing was performed in batches, below shows the processing of 6 .svs files downloaded from the TCGA.

In [101]:
# Paths of all TCGA WSI's:
tcga_paths = glob.glob("tcga/*")
tcga_paths[0:5]

['tcga\\img_crops',
 'tcga\\TCGA-DE-A2OL.svs',
 'tcga\\TCGA-DJ-A13W.svs',
 'tcga\\TCGA-DJ-A13X.svs',
 'tcga\\TCGA-EM-A2CR.svs']

In [39]:
corrupt_files = []

for i in range(len(tcga_paths)):
    c_file = crop_wsi(tcga_paths[i], crop_size = 512, tile_level = 16, 
                  tile_mean = tt_mean, tile_mean_std = tt_mean_std, 
                  tile_std = tt_std, tile_std_std = tt_std_std,
                  save_loc= 'tcga/img_crops', sample_no = 20)
    if c_file is not None:
        corrupt_files.append(c_file)
    print("Processed %s whole slide images..." % (i+1))

Row Processing Progress: 100%|█████████████████████████████████████████████████████████| 27/27 [05:31<00:00, 12.27s/it]


Number of Crops Matching Criteria:  170
Saved 20 cropped images from TCGA-DE-A2OL.
Processed 1 whole slide images...


Row Processing Progress: 100%|█████████████████████████████████████████████████████████| 79/79 [06:04<00:00,  4.62s/it]


Number of Crops Matching Criteria:  1312
Saved 20 cropped images from TCGA-DJ-A13W.
Processed 2 whole slide images...


Row Processing Progress: 100%|█████████████████████████████████████████████████████████| 77/77 [08:14<00:00,  6.42s/it]


...

Number of Crops Matching Criteria:  2514
Saved 20 cropped images from TCGA-EM-A2CR.
Processed 4 whole slide images...


Row Processing Progress: 100%|█████████████████████████████████████████████████████████| 36/36 [03:24<00:00,  5.68s/it]


Number of Crops Matching Criteria:  14
Saved 14 cropped images from TCGA-EM-A3OB.
Processed 5 whole slide images...


Row Processing Progress: 100%|█████████████████████████████████████████████████████████| 78/78 [07:44<00:00,  5.96s/it]


Number of Crops Matching Criteria:  1547
Saved 20 cropped images from TCGA-MK-A84Z.
Processed 6 whole slide images...


# Processing Nikiforov Slides:

In [10]:
def process_nikiforov(slide_nos, slide_dir, 
                     tile_mean, tile_mean_std, tile_std, tile_std_std,
                     dest_folder, sample_no):
    """
    Filter images extracted from WSIs to only include ones that match
    specified criteria (based on ref dataset mean and std). Choose 'X'
    crops at random from that subset and save in destination folder.
    
    slide_nos: a list of slide numbers to process
    """
    for slide in slide_nos:
        # Filepaths of all slide pre-cropped images:
        fpaths = glob.glob('%s%s/*' % (slide_dir, slide))
        
        # Create a list of slides that match required criteria:
        sl_list = []

        for i, file in tqdm(enumerate(fpaths), desc = 'Processing %s Crops' % slide):
            image = Image.open(file)
            img_arr = np.array(image)
            avg = img_arr.mean()
            std = img_arr.std()
            if (avg > tt_mean - tt_mean_std) and (avg < tt_mean + tt_mean_std)\
                        and (std > tt_std - tt_std_std) and (std < tt_std + tt_std_std):
                        sl_list.append(i)
        
        print("Found %s crops matching criteria in Slide %s..." % (len(sl_list), slide))
        
        # Selecting random crops = sample_no
        sample_no = min(len(sl_list), sample_no)
        sel_crops = random.sample(sl_list, sample_no)
        
        # Return filenames of crops selected
        sel_fnames = [fpaths[x] for x in sel_crops]
        
        # Rename and move selected files to different directory:
        for j, file in enumerate(sel_fnames):
            new_fname = '%s/%s/NIK-%s_%s.jpeg' % (slide_dir, slide, slide, j)
            os.rename(file, new_fname)
            shutil.move(new_fname, dest_folder)

In [98]:
slide_nums = ['A102', 'A111', 'A121', 'A123', 'A124', 'A126', 'A127', 'A128', 'A129', 'A130',
                'A131', 'A134', 'A136', 'A137', 'A138', 'A026', 'A027', 'A029', 'A035', 'A036', 'A037', 
                'A041', 'A043', 'A046', 'A047', 'A049', 'A052', 'A056', 'A058', 'A059', 'A060', 'A062', 
                'A073', 'A079', 'A008', 'A080']
slide_dir = "nikiforov/all_imgs/"
dest = 'nikiforov/img_crops/'

In [99]:
process_nikiforov(slide_nos = slide_nums, slide_dir= slide_dir, 
                    tile_mean = tt_mean, tile_mean_std = tt_mean_std, 
                    tile_std = tt_std, tile_std_std = tt_std_std,
                    dest_folder=dest, sample_no=20)

Processing A121 Crops: 11424it [01:44, 109.36it/s]
Processing A123 Crops: 7303it [01:06, 109.11it/s]
Processing A124 Crops: 11088it [01:41, 109.14it/s]
Processing A126 Crops: 12768it [02:04, 102.15it/s]
Processing A127 Crops: 10500it [01:41, 103.36it/s]
Processing A128 Crops: 9156it [01:30, 101.48it/s]
Processing A129 Crops: 9492it [01:32, 102.97it/s]
Processing A130 Crops: 11424it [01:49, 104.67it/s]
Processing A131 Crops: 11424it [01:45, 107.93it/s]
Processing A134 Crops: 9492it [01:27, 108.74it/s]
Processing A136 Crops: 12432it [01:50, 112.09it/s]
Processing A137 Crops: 7000it [01:07, 103.63it/s]
Processing A138 Crops: 10692it [01:36, 110.63it/s]
Processing A026 Crops: 9922it [01:39, 99.43it/s] 
Processing A027 Crops: 9047it [01:31, 98.50it/s] 
Processing A029 Crops: 6868it [01:06, 103.71it/s]
Processing A035 Crops: 7719it [01:13, 104.67it/s]
Processing A036 Crops: 13104it [02:02, 106.73it/s]
Processing A037 Crops: 7381it [01:10, 104.26it/s]
Processing A041 Crops: 7298it [01:11, 102